In [2]:
import pandas as pd
import os
import json
import sys
from os.path import expanduser
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from pysd2cat.data import pipeline
from pysd2cat.analysis import correctness 

from data_converge.util import data_utils as du

%load_ext autoreload
%reload_ext autoreload
%autoreload 2


DATA_CONVERGE_PROJECT="sd2e-project-43"

data_converge_base = os.path.join(expanduser("~"), 'sd2e-projects', DATA_CONVERGE_PROJECT)
experiment_dir = os.path.realpath(os.path.join(data_converge_base, 'test'))
experiment_dir_contents = [os.path.realpath(os.path.join(experiment_dir, x)) for x in os.listdir(experiment_dir)]

experiments = [x for x in  experiment_dir_contents 
               if os.path.isdir(x) and "dc_" in x]

DATA_CONVERGE_CORRECTNESS_DATA_PROJECT="sd2e-project-45"
data_converge_correctness_data_base = os.path.join(expanduser("~"), 'sd2e-projects', DATA_CONVERGE_CORRECTNESS_DATA_PROJECT)

correctness_dir = os.path.join(data_converge_correctness_data_base, 'correctness')
correctness_dir_contents = [os.path.realpath(os.path.join(correctness_dir, x)) for x in os.listdir(correctness_dir)]





The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
def get_analysis_data(experiment_analysis):
    data_converge_id = du.get_data_converge_id(experiment_analysis)
    csv_file = os.path.join(correctness_dir, data_converge_id + ".csv")
    if os.path.exists(csv_file):
        res = pd.read_csv(csv_file)
        data_converge_path = os.path.join(data_converge_base, 'test', data_converge_id)
        res_meta = du.get_meta(data_converge_path, 
                               du.get_record(data_converge_path)).merge(res, on="sample_id", how="inner")
        return res_meta
    else:
        return None

experiment = correctness_dir_contents[0]
print(experiment)
df = get_analysis_data(experiment)


        #out_file_path = os.path.join(expanduser("~"), "tacc-work", "plots")
        #if not os.path.exists(out_file_path):
        #    os.mkdir(out_file_path)
        #out_file = os.path.join(out_file_path, data_converge_id + ".png")
        #do_plot(res_meta, out_file)
  

/work/projects/SD2E-Community/prod/projects/sd2e-project-45/correctness/dc_YeastSTATES-CRISPR-Long-Duration-Time-Series-20191208_20200414085017.csv


In [12]:
strains = df.strain_name.unique()
timepoints = df.timepoint.unique()
inducer_concentrations = df.inducer_concentration.unique()


In [13]:
data_df = du.get_data_and_metadata

array(['SpheroControl', 'beads_spherotech_pps_6K',
       'Negative-(non-fluorescent)-control', 'Inducible-CRISPR-Gate-6',
       'BE-inducible-fluorescent-protein', 'Inducible-CRISPR-Gate-4',
       'Inducible-CRISPR-Gate-7', 'Inducible-CRISPR-Gate-5',
       'Inducible-CRISPR-Gate-2', 'Positive-Control-(yeVenus)',
       'Inducible-CRISPR-Gate-1', 'NOR00-(GFP-positive)-control',
       'Inducible-CRISPR-Gate-3'], dtype=object)